# Updating Data frames

## Data: Palmer Penguins

In [4]:
# import standard libraries 
import pandas as pd
import numpy as np

# import seaborn with standard abbreviation
import seaborn as sns

# use random library to create random numbers
import random

In [6]:
# import data from sns
penguins = sns.load_dataset('penguins')

# look at datamframes head 
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


## Adding a column 

General syntax to add a single column is:

```
df['new_col_name'] = new_column_values
```

`new_column_values` could be:

- a `pd.Series` or `numpy.array` of the same length as the data frame
- a single scalar (single number, single string)

**Example**

Want to create 